# Robustness Dataset Generator

This file generates robustness datasets for the article titled "Model guided design of enhanced bi-stable controllers to  effectively switch cellular states" (Raj K., Wong W. T. Z., Zhang B., & Mahadevan R.) (2022)

This file has a number of dependencies 
- PyDSTool (For bifurcation analysis)
- plotly (for plotting)
- joblib (for parallel computation of results)

# Import Dependencies

In [ ]:
from PyDSTool import * 

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = "none"
import sys
if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML
    HTML("""
         <script>
          var waitForPlotly = setInterval( function() {
          if( typeof(window.Plotly) !== "undefined" ){
          MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
          MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
          clearInterval(waitForPlotly);}}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline as interpolate
from PyDSTool.PyCont.Continuation import PyDSTool_ExistError
from joblib import Parallel, delayed
import multiprocessing
import time
import warnings
import plotly
import pickle


# Define functions that will calculate continuity curves.
- First generate a continuity class object on PyDS tool: This is where the model is defined
- Then compute an equilibrium point for the model and obtain one set of limit points through one parameter continuity.
- Then, using that limit point as a starting point, perform 2 parameter continuity on the protein production rates.
- For both 1 parameter continuity and 2 parameter continuity, the code performs redundant calculations until a valid limit point is obtained. This is needed because the limit point calculation on PyDSTool is unstable in many conditions

In [ ]:
def con_class_gen(params_dict):

    kp_1_par = Par(params_dict['kdeg_1']*10*params_dict['kdiss_2'], 'kp_1')
    kp_2_par = Par(params_dict['kdeg_2']*10*params_dict['kdiss_1'], 'kp_2')
    kdiss_1_par = Par(params_dict['kdiss_1'], 'k_diss_1')
    kdiss_2_par = Par(params_dict['kdiss_2'], 'k_diss_2')
    kdeg_1_par = Par(params_dict['kdeg_1'], 'k_deg_1')
    kdeg_2_par = Par(params_dict['kdeg_2'], 'k_deg_2')
    n_1_par = Par(params_dict['n_1'], 'n_1')
    n_2_par = Par(params_dict['n_2'], 'n_2')

    lac_i = Var('lac_i')
    tet_r = Var('tet_r')
    dlaci_dt = kp_1_par/(1+(tet_r/kdiss_1_par)**n_1_par) - kdeg_1_par*lac_i
    dtetr_dt = kp_2_par/(1+(lac_i/kdiss_2_par)**n_2_par) - kdeg_2_par*tet_r

    DSargs = args(name='Simple Toggle Equilibrium Model')
    DSargs.pars = [kp_1_par, kp_2_par, kdiss_1_par, kdiss_2_par, kdeg_1_par, kdeg_2_par, n_1_par, n_2_par]
    DSargs.varspecs = args(lac_i=dlaci_dt,
                           tet_r=dtetr_dt)

    DSargs.fnspecs = {'Jacobian': (['t','lac_i','tet_r'],
                                    """[[-k_deg_1, -(kp_1*n_1/k_diss_1**n_1)*(tet_r**(n_1-1)/(1+(tet_r/k_diss_1)**n_1)**2) ],
                                        [ -(kp_2*n_2/k_diss_2**n_2)*(lac_i**(n_2-1)/(1+(lac_i/k_diss_2)**n_2)**2),  -k_deg_2 ]]""")}
    DSargs.ics = args(lac_i=10*params_dict['kdiss_2']/params_dict['kdiss_1'], tet_r=params_dict['kdiss_2']/params_dict['kdiss_1']/100)

    DSargs.xdomain = {'lac_i': [0, 100000000],
                      'tet_r': [0, 100000000]}
    DSargs.tdomain = [0, 1000000000]
    DSargs.pdomain = {'kp_1': [0, 100000000], 'kp_2': [0, 100000000],
                     'k_diss_1': [0.00, 1000000000],
                     'k_diss_2': [0.000, 100000000],
                     'k_deg_1': [0.0, 100000000],
                     'k_deg_2': [0.0, 1000000000],
                     'n_1':[1, 8],
                    'n_2':[1, 8]}

    ode_sys = Generator.Vode_ODEsystem(DSargs)
    con_ode_sys = ContClass(ode_sys)
    return con_ode_sys

def get_limit_points(con_ode_sys):
    try:
        LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
        LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
        LP3 = con_ode_sys['EQ1'].getSpecialPoint('LP3')
        LP4 = con_ode_sys['EQ1'].getSpecialPoint('LP4')
    except:
        return []
    LP_list = [LP for LP in [LP1, LP2, LP3, LP4] if LP and LP[con_ode_sys['EQ1'].freepars[0]]>0]
    unique_lp_vals = list(set([np.round(LP[con_ode_sys['EQ1'].freepars[0]], 5) for LP in LP_list]))
    final_lp_list = []
    for LP in LP_list:
        if np.round(LP[con_ode_sys['EQ1'].freepars[0]], 5) in unique_lp_vals:
            unique_lp_vals.remove(np.round(LP[con_ode_sys['EQ1'].freepars[0]], 5))
            final_lp_list.append(LP.labels['LP']['name'])
    LP_list = final_lp_list

    LP_list = sorted(LP_list, key=lambda x: con_ode_sys['EQ1'].getSpecialPoint(x)[con_ode_sys['EQ1'].freepars[0]])
    return LP_list

def generate_equilibrium_curve(params_dict, threshold):
    con_ode_sys = con_class_gen(params_dict)
    PCargs = args(name='EQ1',type='EP-C', force=True)
    if con_ode_sys.model.pars['kp_1'] > params_dict['kp_2']: #Determine which kdeg is higher and change corresponding kp because that gives finer variations in limit point determination
        PCargs.freepars = ['kp_1']
    else:
        PCargs.freepars = ['kp_2']


    PCargs.StepSize = 1e-3
    PCargs.MaxNumPoints = 500
    PCargs.MaxStepSize = max(params_dict['kdeg_1'], params_dict['kdeg_2'])/10*min(params_dict['kdiss_1'], params_dict['kdiss_2'])*10000
    PCargs.MinStepSize = 1e-7
    PCargs.LocBifPoints = ['LP']
    PCargs.StopAtPoints = ['LP']
    con_ode_sys.newCurve(PCargs)

    try:
        con_ode_sys['EQ1'].backward()
        con_ode_sys['EQ1'].backward()
        LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
        LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
    except:
        print('fail')
        LP1 = None
        LP2 = None

    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('enter')
        freepars = {'kp_1': 'kp_2', 'kp_2': 'kp_1'}
        PCargs.freepars = [freepars[PCargs.freepars[0]]]
        con_ode_sys.newCurve(PCargs)
        try:
            con_ode_sys['EQ1'].backward()
            con_ode_sys['EQ1'].backward()
            LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
            LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
        except:
            print('fail')
            LP1 = None
            LP2 = None

    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('enter2')
        PCargs.MaxStepSize = PCargs.MaxStepSize/10
        con_ode_sys.newCurve(PCargs)
        try:
            con_ode_sys['EQ1'].backward()
            con_ode_sys['EQ1'].backward()
            LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
            LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
        except:
            print('fail')
            LP1 = None
            LP2 = None


    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('OH NO')
        PCargs.initpoint = {'lac_i': 100,
                            'tet_r': 0.0001,
                            PCargs.freepars[0]:0.00001}
        con_ode_sys.newCurve(PCargs)
        try:
            con_ode_sys['EQ1'].backward()
            con_ode_sys['EQ1'].backward()
            LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
            LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
        except:
            print('fail')
            LP1 = None
            LP2 = None

    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('Still no')
        PCargs.MaxStepSize = 1
        freepars = {'kp_1': 'kp_2', 'kp_2': 'kp_1'}
        PCargs.freepars = [freepars[PCargs.freepars[0]]]
        con_ode_sys.newCurve(PCargs)
        PCargs.initpoint = {'lac_i': 1,
                    'tet_r': .001,
                    PCargs.freepars[0]:0.0001}
        con_ode_sys.newCurve(PCargs)
        try:
            con_ode_sys['EQ1'].backward()
            con_ode_sys['EQ1'].backward()
            LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
            LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
        except:
            print('fail')
            LP1 = None
            LP2 = None
            
    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('Still no')
        PCargs.MaxStepSize = 10
        freepars = {'kp_1': 'kp_2', 'kp_2': 'kp_1'}
        PCargs.freepars = [freepars[PCargs.freepars[0]]]
        con_ode_sys.newCurve(PCargs)
        PCargs.initpoint = {'lac_i': 1,
                    'tet_r': .001,
                    PCargs.freepars[0]:0.0001}
        con_ode_sys.newCurve(PCargs)
        try:
            con_ode_sys['EQ1'].backward()
            con_ode_sys['EQ1'].backward()
            LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
            LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
        except:
            print('fail')
            LP1 = None
            LP2 = None


    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('give up')
        PCargs.MaxStepSize = .1
        freepars = {'kp_1': 'kp_2', 'kp_2': 'kp_1'}
        PCargs.freepars = [freepars[PCargs.freepars[0]]]
        con_ode_sys.newCurve(PCargs)
        PCargs.initpoint = {'lac_i': 1,
                    'tet_r': .001,
                    PCargs.freepars[0]:0.0001}
        con_ode_sys.newCurve(PCargs)
        try:
            con_ode_sys['EQ1'].backward()
            con_ode_sys['EQ1'].backward()
            LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
            LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
        except:
            print('fail')
            LP1 = None
            LP2 = None

    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('final try')
        PCargs.MaxStepSize = max(params_dict['kdeg_1'], params_dict['kdeg_2'])/10*min(params_dict['kdiss_1'], params_dict['kdiss_2'])*10000
        PCargs.freepars = [freepars[PCargs.freepars[0]]]
        con_ode_sys.newCurve(PCargs)
        PCargs.initpoint = {'lac_i': 1,
                    'tet_r': .001,
                    PCargs.freepars[0]:0.0001}
        con_ode_sys.newCurve(PCargs)
        try:
            con_ode_sys['EQ1'].backward()
            con_ode_sys['EQ1'].backward()
            LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
            LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
        except:
            print('fail')
            LP1 = None
            LP2 = None

    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('final try')
        PCargs.MaxStepSize = max(params_dict['kdeg_1'], params_dict['kdeg_2'])/10*min(params_dict['kdiss_1'], params_dict['kdiss_2'])*10000
        PCargs.freepars = [freepars[PCargs.freepars[0]]]
        con_ode_sys.newCurve(PCargs)
        PCargs.initpoint = {'lac_i': 1,
                    'tet_r': .001,
                    PCargs.freepars[0]:0.0001}
        con_ode_sys.newCurve(PCargs)
        try:
            con_ode_sys['EQ1'].backward()
            con_ode_sys['EQ1'].backward()
            LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
            LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
        except:
            print('fail')
            LP1 = None
            LP2 = None


    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
            print('hmmm')

            PCargs.initpoint = {'lac_i': 10000,
                        'tet_r': 0.1,
                        PCargs.freepars[0]:0.0001}
            con_ode_sys.newCurve(PCargs)
            try:
                con_ode_sys['EQ1'].backward()
                con_ode_sys['EQ1'].backward()
                LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
                LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
            except:
                print('fail')
                LP1 = None
                LP2 = None

    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('whatever')
        freepars = {'kp_1': 'kp_2', 'kp_2': 'kp_1'}
        PCargs.freepars = [freepars[PCargs.freepars[0]]]
        con_ode_sys.newCurve(PCargs)
        PCargs.initpoint = {'lac_i': 0.001,
                    'tet_r': 0.1,
                    PCargs.freepars[0]:0.0001}
        con_ode_sys.newCurve(PCargs)
    try:
        con_ode_sys['EQ1'].forward()
        con_ode_sys['EQ1'].forward()
        con_ode_sys['EQ1'].forward()
        LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
        LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
    except:
        print('fail')
        LP1 = None
        LP2 = None

    LP_list = get_limit_points(con_ode_sys)
    if len(LP_list) < 2:
        print('entered again')
        try:
            con_ode_sys['EQ1'].backward()
        except:
            print('No')
        try:
            con_ode_sys['EQ1'].forward()
        except:
            print('No')
        LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
        LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')

    LP_list = get_limit_points(con_ode_sys)
    if LP_list:
        if con_ode_sys['EQ1'].freepars[0]=='kp_1':
            LP_list = list(reversed(LP_list))
        init_point = LP_list[0]
    else:
        init_point = None
        
    return con_ode_sys, init_point

# This function processes the continuity curbves and generates robustness values as the fraction of area bound by the continuity curves
def process_continuation_results(x=[], y=[], params_dict={}, threshold=1000):
    min_x = min(x)
    min_y = min(y)

    if (y[0] - min(y))/(x[0] - min(x)) > (y[-1] - min(y))/(x[-1] - min(x)):#Condition to check if upper curve begins first
        upper_x = np.sort(x[:np.argmin(x)+1])
        upper_y = np.sort(y[:np.argmin(x)+1])
        lower_x = np.sort(x[np.argmin(x):])
        lower_y = np.sort(y[np.argmin(x):])
    else:
        lower_x = np.sort(x[:np.argmin(x)+1])
        lower_y = np.sort(y[:np.argmin(x)+1])
        upper_x = np.sort(x[np.argmin(x):])
        upper_y = np.sort(y[np.argmin(x):])

    raw_lower_x = lower_x
    raw_lower_y = lower_y
    raw_upper_x = upper_x
    raw_upper_y = upper_y

    #Clearing any duplicate entries in x or y - helps clear up probs with interpolation
    u, indices = np.unique(lower_x, return_index=True)
    lower_x = lower_x[indices]
    lower_y = lower_y[indices]
    u, indices = np.unique(lower_y, return_index=True)
    lower_x = lower_x[indices]
    lower_y = lower_y[indices]

    u, indices = np.unique(upper_x, return_index=True)
    upper_x = upper_x[indices]
    upper_y = upper_y[indices]
    u, indices = np.unique(upper_y, return_index=True)
    upper_x = upper_x[indices]
    upper_y = upper_y[indices]

    try:
        lower_interp = interpolate(lower_x, lower_y, k=3)
        upper_interp = interpolate(upper_x, upper_y, k=3)
    except:
        results_dict = {'params': params_dict,
            'robustness': 0,
            'lower_x': [],
            'lower_y': [],
            'upper_x': [],
            'upper_y': [],
            'raw_lower_x': [],
            'raw_lower_y': [],
            'raw_upper_x': [],
            'raw_upper_y': [],
            'raw_x': x,
            'raw_y': y,
            'status': False}
        return results_dict


    if lower_interp(threshold) < 0:
        lower_x = lower_x[:-1]
        lower_y = lower_y[:-1]

    if upper_interp(threshold) < 0:
        upper_x = upper_x[:-1]
        upper_y = upper_y[:-1]

    if upper_interp(threshold) < threshold and upper_interp(threshold) > 0: #Upper curve does not hit upper threshold -> need to cut short everything at x(kp1)=100

        lower_interp = interpolate(lower_x, lower_y, k=3)
        upper_interp = interpolate(upper_x, upper_y, k=3)

        lower_x = np.linspace(min(lower_x), threshold, 1000)
        lower_y = lower_interp(lower_x)
        upper_x = np.linspace(min(upper_x), threshold, 1000)
        upper_y = upper_interp(upper_x)

    elif lower_interp(threshold) > threshold: #Lower curve hits upper threshold -> Need to cut short everything at y(kp2)=100
        lower_interp = interpolate(lower_y, lower_x, k=3)
        upper_interp = interpolate(upper_y, upper_x, k=3)

        if lower_interp(threshold) > threshold:#Irregular interpolation. Deal by keeping this but truncating appropriately
            lower_y = np.linspace(min(lower_y), threshold, 1000)
            lower_x = lower_interp(np.linspace(min(lower_y), threshold, 1000))
            u, indices = np.unique(lower_x, return_index=True)
            lower_x = lower_x[indices]
            lower_y = lower_y[indices]
            lower_interp = interpolate(lower_x, lower_y, 
                                       k=3)
            lower_x = np.linspace(min(lower_x), threshold, 1000)
            lower_y = lower_interp(lower_x)
        else:

            lower_y = np.linspace(min(lower_y), threshold, 1000)
            lower_x = lower_interp(lower_y)

        if upper_interp(threshold) > lower_interp(threshold):#irregular case
            upper_interp = interpolate(upper_x, upper_y, k=3)
            upper_x = np.linspace(min(upper_x), 5*max(upper_x), 1000)
            upper_y = upper_interp(upper_x)
            u, indices = np.unique(upper_y, return_index=True)
            upper_x = upper_x[indices]
            upper_y = upper_y[indices]
            upper_interp = interpolate(upper_y, upper_x, k=3)  


        upper_y = np.linspace(min(upper_y), threshold, 1000)
        upper_x = upper_interp(upper_y)
        upper_y = np.append(upper_y, [threshold for i in range(100)])
        upper_x = np.append(upper_x, np.linspace(max(upper_x), max(lower_x), 100))

    elif lower_interp(threshold) < threshold: # A normal continuity curve - lower curve intersects x=threshold and upper curve intersects y=threshold
        lower_interp = interpolate(lower_y, lower_x, k=3)
        upper_interp = interpolate(upper_x, upper_y, k=3)


        lower_y = np.linspace(min(lower_y), threshold, 1000)
        lower_x = lower_interp(lower_y)
        u, indices = np.unique(lower_x, return_index=True)
        lower_x = lower_x[indices]
        lower_y = lower_y[indices]
        lower_interp = interpolate(lower_x, 
                               lower_y, 
                               k=3)
        lower_x = np.linspace(min(lower_x), threshold, 1000)
        lower_y = lower_interp(lower_x)

        if upper_interp(threshold)<0 and lower_interp(threshold)>0:

            upper_x = np.linspace(min(upper_x), 5*max(upper_x), 1000)
            upper_y = upper_interp(upper_x)
            u, indices = np.unique(upper_y, return_index=True)
            upper_x = upper_x[indices]
            upper_y = upper_y[indices]
            upper_interp = interpolate(upper_y, upper_x, k=3)  
            upper_y = np.linspace(min(upper_y), 2*max(upper_y), 1000)
            upper_x = upper_interp(upper_y)
            u, indices = np.unique(upper_x, return_index=True)
            upper_x = upper_x[indices]
            upper_y = upper_y[indices]
            upper_interp = interpolate(upper_x, upper_y, k=3)

                
            if upper_interp(threshold) > threshold:
                upper_x = np.linspace(min(upper_x), threshold, 1000)
                upper_y = upper_interp(upper_x)
                upper_y = np.array([element if element<threshold else threshold for element in upper_y])
            if upper_interp(threshold) < threshold and upper_interp(threshold):
                upper_x = np.linspace(min(upper_x), 5*max(upper_x), 1000)
                upper_y = upper_interp(upper_x)
                u, indices = np.unique(upper_y, return_index=True)
                upper_x = upper_x[indices]
                upper_y = upper_y[indices]
                upper_interp = interpolate(upper_y, upper_x, k=3)
                upper_y = np.linspace(min(upper_y), threshold, 1000)
                upper_x = upper_interp(upper_y)
                upper_x = np.append(upper_x, np.linspace(max(upper_x), threshold, 100)[1:])
                upper_y = np.append(upper_y, [threshold]*len(np.linspace(max(upper_x), threshold, 100)[1:]))

            if (upper_x[-1]<0) or (upper_y[-1]<0):

                upper_interp = interpolate(raw_upper_x, raw_upper_y, k=3)
                upper_x = np.linspace(min(raw_upper_x), 5*max(raw_upper_x), 1000)
                upper_y = upper_interp(upper_x)
                u, indices = np.unique(upper_y, return_index=True)
                upper_x = upper_x[indices]
                upper_y = upper_y[indices]
                upper_x = np.linspace(min(upper_x), 5*max(upper_x), 1000)
                upper_y = upper_interp(upper_x)
                u, indices = np.unique(upper_y, return_index=True)
                upper_x = upper_x[indices]
                upper_y = upper_y[indices]
                upper_interp = interpolate(upper_y, upper_x, k=3) 
                upper_y = np.linspace(min(upper_y), threshold, 1000)
                upper_x = upper_interp(upper_y)
                upper_x = np.append(upper_x, np.linspace(max(upper_x), threshold, 100)[1:])
                upper_y = np.append(upper_y, [threshold]*len(np.linspace(max(upper_x), threshold, 100)[1:]))
        else:
            upper_x = np.linspace(min(upper_x), threshold, 1000)
            upper_y = upper_interp(upper_x)
            upper_y = np.array([element if element<threshold else threshold for element in upper_y])

    else:
        results_dict = {'params': params_dict,
            'robustness': 0,
            'lower_x': [],
            'lower_y': [],
            'upper_x': [],
            'upper_y': [],
            'raw_lower_x': [],
            'raw_lower_y': [],
            'raw_upper_x': [],
            'raw_upper_y': [],
            'raw_x': x,
            'raw_y': y,
            'status': True}
        return results_dict


    robustness = np.trapz(upper_y, upper_x) - np.trapz(lower_y, lower_x)
    if robustness < 0:
        robustness = 0



    results_dict = {'params': params_dict,
                    'robustness': robustness,
                    'lower_x': lower_x,
                    'lower_y': lower_y,
                    'upper_x': upper_x,
                    'upper_y': upper_y,
                    'raw_lower_x': raw_lower_x,
                    'raw_lower_y': raw_lower_y,
                    'raw_upper_x': raw_upper_x,
                    'raw_upper_y': raw_upper_y,
                    'raw_x': x,
                    'raw_y': y,
                    'status': True}

    return results_dict


def simple_toggle_robustness_repeat(params_dict, threshold):
    
    con_ode_sys, init_point = generate_equilibrium_curve(params_dict, threshold)
    PCargs = args(name='robustness', type='LP-C', force=True)
    LP_list = get_limit_points(con_ode_sys)
    if LP_list:
        PCargs.initpoint = 'EQ1:'+init_point
        PCargs.freepars = ['kp_1','kp_2']
        PCargs.MaxNumPoints = 500
        PCargs.MinStepSize = 0.0001
        PCargs.MaxStepSize = con_ode_sys['EQ1'].getSpecialPoint(init_point)[con_ode_sys['EQ1'].freepars[0]]/100
        if PCargs.MaxStepSize < 10:
            PCargs.MaxStepSize = 10
        attempts = 0
        success = False
        while not success and attempts < 3:
            attempts += 1
            print('attempts= ', attempts)
            try:
                con_ode_sys.newCurve(PCargs)
                try:
                    con_ode_sys['robustness'].forward()
                    con_ode_sys['robustness'].backward()
                except:
                    con_ode_sys['robustness'].backward()
                    con_ode_sys['robustness'].forward()
                success=True
            except:
                if len(LP_list) > 1:
                    init_point = [LP for LP in LP_list if LP!=init_point][0]
                    PCargs.initpoint = 'EQ1:'+init_point


                PCargs.MaxNumPoints = int(PCargs.MaxNumPoints * 1.5)
                PCargs.MaxStepSize = con_ode_sys['EQ1'].getSpecialPoint(init_point)[con_ode_sys['EQ1'].freepars[0]]/100
                if PCargs.MaxStepSize < 10:
                    PCargs.MaxStepSize = 20
                PCargs.MinStepSize = PCargs.MinStepSize / 10

                x = []
                y = []
                continue
            x = con_ode_sys['robustness'].sol['kp_1']
            y = con_ode_sys['robustness'].sol['kp_2']
            print("x=", x, 'y= ',y)
            if len(x) > 2 and len(y) > 2:
                if (x[0] >= x[1]) or (y[0] >= y[1]):
                    x = x[1:]
                    y = y[1:]
                if (x[-1] <= x[-2]) or (y[-1] <= y[-2]):
                    y = y[:-1]
                    x = x[:-1]
                print(x[0], y[0])
                print(x[-1], y[-1])
                gradients = np.gradient(y,x)
                
                if np.argmin(x) in range(1,len(x)-1) or np.argmin(y) in range(1, len(y)-1):
                    
                    if (min(x) > threshold) or (min(y) > threshold):
                        print('broke')
                        break
                print("Beginning Gradient: ", gradients[0]/gradients[1])
                print("Ending Gradient: ", gradients[-1]/gradients[-2])
                if (gradients[0]/gradients[1]<0.95 or gradients[0]/gradients[1]>1.05): #This means closed curve at beginning. Trim first point
                    print('closed at beginning')
                    x = x[1:]
                    y = y[1:]
                    PCargs.MaxNumPoints = int(PCargs.MaxNumPoints * 2)
                    PCargs.MaxStepSize = PCargs.MaxStepSize/10
                    PCargs.MinStepSize = PCargs.MinStepSize / 10
                    if attempts == 3:
                        success = True
                    else:
                        print('repeating because closed at beginning')
                        success = False
                    continue

                elif (gradients[-1]/gradients[-2]<0.95 or gradients[-1]/gradients[-2]>1.05):
                    print('closed at end')

                    if attempts == 3:
                        success = True
                    else:
                        print('repeating because closed at end')
                        success = False
                    x = x[:-1]
                    y = y[:-1]
                    PCargs.MaxNumPoints = int(PCargs.MaxNumPoints * 2)
                    PCargs.MaxStepSize = PCargs.MaxStepSize/10
                    PCargs.MinStepSize = PCargs.MinStepSize / 10
                    continue


                elif np.argmin(x) in range(0,3) or np.argmin(x) in range(len(x)-3,len(x)):
                    PCargs.MaxNumPoints = int(PCargs.MaxNumPoints*1.5)
                    PCargs.MaxStepSize = max(min(x), min(y))/100
                    if PCargs.MaxStepSize < 10:
                        PCargs.MaxStepSize = 20 
                    PCargs.MinStepSize = PCargs.MinStepSize / 10
                    x = []
                    y = []
                    print('repeating because min is too close to ends of list')
                    success = False

                elif not ((x[0] > threshold or y[0] > threshold) and 
                            (x[-1] > threshold or y[-1] > threshold)):

                    PCargs.MaxNumPoints = int(PCargs.MaxNumPoints*1.5)
                    PCargs.MaxStepSize = max(min(x), min(y))/100
                    if PCargs.MaxStepSize < 10:
                        PCargs.MaxStepSize = 20
                        
                    PCargs.MinStepSize = PCargs.MinStepSize / 10
                    if attempts == 3:
                        success = True
                    else:
                        print('repeating because curve is not exactly complete')
                        success = False

        if len(x)!=0 and len(y)!=0:
            if not (min(x)<=threshold and min(y)<=threshold):
                
                success=False
                
        if success:
            results_dict = process_continuation_results(x, y, params_dict)
            
        else:
            results_dict = {'params': params_dict,
                            'robustness': 0,
                            'lower_x': [],
                            'lower_y': [],
                            'upper_x': [],
                            'upper_y': [],
                            'raw_lower_x': [],
                            'raw_lower_y': [],
                            'raw_upper_x': [],
                            'raw_upper_y': [],
                            'raw_x': x,
                            'raw_y': y,
                            'status': False}
    else:
        results_dict = {'params': params_dict,
                'robustness': 0,
                'lower_x': [],
                'lower_y': [],
                'upper_x': [],
                'upper_y': [],
                'raw_lower_x': [],
                'raw_lower_y': [],
                'raw_upper_x': [],
                'raw_upper_y': [],
                'raw_x': [],
                'raw_y': [],
                'status': False}
    return results_dict

    
def simple_toggle_robustness(params_dict, threshold):
    
    con_ode_sys, init_point = generate_equilibrium_curve(params_dict, threshold)
    PCargs = args(name='robustness', type='LP-C', force=True)
    LP_list = get_limit_points(con_ode_sys)
     
    if LP_list:
        PCargs.initpoint = 'EQ1:'+init_point
        PCargs.freepars = ['kp_1','kp_2']
        PCargs.MaxNumPoints = 500
        PCargs.MinStepSize = 0.0001
        PCargs.MaxStepSize = con_ode_sys['EQ1'].getSpecialPoint(init_point)[con_ode_sys['EQ1'].freepars[0]]/100
        if PCargs.MaxStepSize < 10:
            PCargs.MaxStepSize = 10
        attempts = 0
        success = False
        while not success and attempts < 3:
            attempts += 1
            print('attempts= ', attempts)
            try:
                con_ode_sys.newCurve(PCargs)
                try:
                    con_ode_sys['robustness'].forward()
                    con_ode_sys['robustness'].backward()
                except:
                    con_ode_sys['robustness'].backward()
                    con_ode_sys['robustness'].forward()
                success=True
            except:
                if len(LP_list) > 1:
                    init_point = [LP for LP in LP_list if LP!=init_point][0]
                    PCargs.initpoint = 'EQ1:'+init_point

                PCargs.MaxNumPoints = int(PCargs.MaxNumPoints * 1.5)
                PCargs.MaxStepSize = con_ode_sys['EQ1'].getSpecialPoint(init_point)[con_ode_sys['EQ1'].freepars[0]]/100
                if PCargs.MaxStepSize < 10:
                    PCargs.MaxStepSize = 20
                PCargs.MinStepSize = PCargs.MinStepSize / 10

                x = []
                y = []
                continue
            x = con_ode_sys['robustness'].sol['kp_1']
            y = con_ode_sys['robustness'].sol['kp_2']
        

            if len(x) > 2 and len(y) > 2:
                if (x[0] >= x[1]) or (y[0] >= y[1]):
                    x = x[1:]
                    y = y[1:]
                if (x[-1] <= x[-2]) or (y[-1] <= y[-2]):
                    y = y[:-1]
                    x = x[:-1]
                print(x[0], y[0])
                print(x[-1], y[-1])
                gradients = np.gradient(y,x)
                if np.argmin(x) in range(1,len(x)-1) or np.argmin(y) in range(1, len(y)-1):
                    if min(x) > threshold or min(y) > threshold:
                        print('broke')
                        break
                if (gradients[0]/gradients[1]<0.95 or gradients[0]/gradients[1]>1.05): #This means closed curve at beginning. Trim first point
                    x = x[1:]
                    y = y[1:]
                    print('entered closed curve at beginning')
                    if len(LP_list) > 1:
                        init_point = [LP for LP in LP_list if LP!=init_point][0]
                        PCargs.initpoint = 'EQ1:'+init_point
                        PCargs.MaxStepSize = con_ode_sys['EQ1'].getSpecialPoint(init_point)[con_ode_sys['EQ1'].freepars[0]]/100
                    else:
                        PCargs.MaxNumPoints = int(PCargs.MaxNumPoints * 1.5)
                        PCargs.MaxStepSize = PCargs.MaxStepSize/10
                    
                    if PCargs.MaxStepSize < 10:
                        PCargs.MaxStepSize = 20
                    PCargs.MinStepSize = PCargs.MinStepSize / 10
                    if attempts == 3:
                        success = True
                    else:
                        success = False
                    continue

                elif (gradients[-1]/gradients[-2]<0.95 or gradients[-1]/gradients[-2]>1.05):
                    print('entered closed curve at end')

                    if attempts == 3:
                        success = True
                    else:
                        success = False
                    x = x[:-1]
                    y = y[:-1]

                    if len(LP_list) > 1:
                        init_point = [LP for LP in LP_list if LP!=init_point][0]
                        PCargs.initpoint = 'EQ1:'+init_point
                        PCargs.MaxStepSize = con_ode_sys['EQ1'].getSpecialPoint(init_point)[con_ode_sys['EQ1'].freepars[0]]/100
                    else:
                        PCargs.MaxNumPoints = int(PCargs.MaxNumPoints * 1.5)
                        PCargs.MaxStepSize = PCargs.MaxStepSize/10
                    
                    if PCargs.MaxStepSize < 10:
                        PCargs.MaxStepSize = 20
                    PCargs.MinStepSize = PCargs.MinStepSize / 10

                    continue


                elif np.argmin(x) in range(0,3) or np.argmin(x) in range(len(x)-3,len(x)):
                    PCargs.MaxNumPoints = int(PCargs.MaxNumPoints*1.5)
                    PCargs.MaxStepSize = max(min(x), min(y))/100
                    if PCargs.MaxStepSize < 10:
                        PCargs.MaxStepSize = 20
                    PCargs.MinStepSize = PCargs.MinStepSize / 10

                    x = []
                    y = []
                    success = False

                elif not ((x[0] > threshold or y[0] > threshold) and 
                            (x[-1] > threshold or y[-1] > threshold)):
                    print(x[0], y[0])
                    print(x[-1], y[-1])
                    PCargs.MaxNumPoints = int(PCargs.MaxNumPoints*1.5)
                    PCargs.MaxStepSize = max(min(x), min(y))/100
                    if PCargs.MaxStepSize < 10:
                        PCargs.MaxStepSize = 20
                    PCargs.MinStepSize = PCargs.MinStepSize / 10
                    print(PCargs.MaxNumPoints, PCargs.MaxStepSize, PCargs.MinStepSize)
                    if attempts == 3:
                        success = True
                    else:
                        success = False
                        
        if len(x)!=0 and len(y)!=0:
            if not (min(x)<=threshold and min(y)<=threshold):
                success=False
                
        if success:
            results_dict = process_continuation_results(x, y, params_dict)
            
        else:
            results_dict = {'params': params_dict,
                            'robustness': 0,
                            'lower_x': [],
                            'lower_y': [],
                            'upper_x': [],
                            'upper_y': [],
                            'raw_lower_x': [],
                            'raw_lower_y': [],
                            'raw_upper_x': [],
                            'raw_upper_y': [],
                            'raw_x': x,
                            'raw_y': y,
                            'status': False}
    else:
        results_dict = {'params': params_dict,
                'robustness': 0,
                'lower_x': [],
                'lower_y': [],
                'upper_x': [],
                'upper_y': [],
                'raw_lower_x': [],
                'raw_lower_y': [],
                'raw_upper_x': [],
                'raw_upper_y': [],
                'raw_x': [],
                'raw_y': [],
                'status': False}
    return results_dict
    
def simple_toggle_robustness_final(params_dict, threshold):
    
    con_ode_sys, init_point = generate_equilibrium_curve(params_dict, threshold)
    PCargs = args(name='robustness', type='LP-C', force=True)
    LP_list = get_limit_points(con_ode_sys)
    if LP_list:
        PCargs.initpoint = 'EQ1:'+init_point
        PCargs.freepars = ['kp_1','kp_2']
        PCargs.MaxNumPoints = 500
        PCargs.MinStepSize = 0.0001
        PCargs.MaxStepSize = con_ode_sys['EQ1'].getSpecialPoint(init_point)[con_ode_sys['EQ1'].freepars[0]]/100
        if PCargs.MaxStepSize < 10:
            PCargs.MaxStepSize = 10
        attempts = 0
        success = False
        while not success and attempts < 3:
            attempts += 1
            print('attempts= ', attempts)
            try:
                con_ode_sys.newCurve(PCargs)
                try:
                    con_ode_sys['robustness'].forward()
                    con_ode_sys['robustness'].backward()
                except:
                    con_ode_sys['robustness'].backward()
                    con_ode_sys['robustness'].forward()
                success=True
            except:
                print('entered except')
                if len(LP_list) > 1:
                    init_point = [LP for LP in LP_list if LP!=init_point][0]
                    PCargs.initpoint = 'EQ1:'+init_point

                PCargs.MaxNumPoints = int(PCargs.MaxNumPoints * 1)
                PCargs.MaxStepSize = con_ode_sys['EQ1'].getSpecialPoint(init_point)[con_ode_sys['EQ1'].freepars[0]]/100
                if PCargs.MaxStepSize < 10:
                    PCargs.MaxStepSize = 20
                PCargs.MinStepSize = PCargs.MinStepSize / 10

                x = []
                y = []
                continue
            x = con_ode_sys['robustness'].sol['kp_1']
            y = con_ode_sys['robustness'].sol['kp_2']

        if len(x)!=0 and len(y)!=0:
            if not (min(x)<=threshold and min(y)<=threshold):
                success=False
                
        if success:
            results_dict = process_continuation_results(x, y, params_dict)
            
        else:
            results_dict = {'params': params_dict,
                            'robustness': 0,
                            'lower_x': [],
                            'lower_y': [],
                            'upper_x': [],
                            'upper_y': [],
                            'raw_lower_x': [],
                            'raw_lower_y': [],
                            'raw_upper_x': [],
                            'raw_upper_y': [],
                            'raw_x': x,
                            'raw_y': y,
                            'status': False}
    else:
        results_dict = {'params': params_dict,
                'robustness': 0,
                'lower_x': [],
                'lower_y': [],
                'upper_x': [],
                'upper_y': [],
                'raw_lower_x': [],
                'raw_lower_y': [],
                'raw_upper_x': [],
                'raw_upper_y': [],
                'raw_x': [],
                'raw_y': [],
                'status': False}
    return results_dict




def new_simple_toggle_robustness(params_dict, threshold):
    
    
    
    con_ode_sys, init_point = generate_equilibrium_curve(params_dict, threshold)
    PCargs = args(name='robustness', type='LP-C', force=True)
    

    LP1 = con_ode_sys['EQ1'].getSpecialPoint('LP1')
    LP2 = con_ode_sys['EQ1'].getSpecialPoint('LP2')
    LP3 = con_ode_sys['EQ1'].getSpecialPoint('LP3')
    LP4 = con_ode_sys['EQ1'].getSpecialPoint('LP4')
    LP_list = [LP for LP in [LP1, LP2, LP3, LP4] if LP and LP[con_ode_sys['EQ1'].freepars[0]]>0]
    unique_lp_vals = list(set([np.round(LP[con_ode_sys['EQ1'].freepars[0]], 5) for LP in LP_list]))
    final_lp_list = []
    for LP in LP_list:
        if np.round(LP[con_ode_sys['EQ1'].freepars[0]], 5) in unique_lp_vals:
            unique_lp_vals.remove(np.round(LP[con_ode_sys['EQ1'].freepars[0]], 5))
            final_lp_list.append(LP.labels['LP']['name'])
    LP_list = final_lp_list
    LP_list = sorted(LP_list, key=lambda x: con_ode_sys['EQ1'].getSpecialPoint(x)[con_ode_sys['EQ1'].freepars[0]])

    
    if con_ode_sys['EQ1'].freepars[0]=='kp_1':
        LP_list = list(reversed(LP_list))

    if len(LP_list) == 2:

        x = [None,None]
        y = [None,None]
        for i, LP in enumerate(LP_list):
            PCargs.initpoint = 'EQ1:'+LP
            PCargs.freepars = ['kp_1','kp_2']
            PCargs.MaxNumPoints = 100
            PCargs.MinStepSize = 0.00001
            PCargs.MaxStepSize = con_ode_sys['EQ1'].getSpecialPoint(LP)[con_ode_sys['EQ1'].freepars[0]]
            if PCargs.MaxStepSize >0.1:
                PCargs.MaxStepSize = 0.1
            attempts = 0
            success = False
            while not success and attempts < 3:
                attempts += 1
                print('attempts= ', attempts)
                try:
                    con_ode_sys.newCurve(PCargs)
                    try:
                        con_ode_sys['robustness'].forward()
                        con_ode_sys['robustness'].backward()
                    except:
                        con_ode_sys['robustness'].backward()
                        con_ode_sys['robustness'].forward()
                    success=True
                except:
                    results_dict = {'params': params_dict,
                        'robustness': 0,
                        'lower_x': [],
                        'lower_y': [],
                        'upper_x': [],
                        'upper_y': [],
                        'raw_lower_x': [],
                        'raw_lower_y': [],
                        'raw_upper_x': [],
                        'raw_upper_y': [],
                        'raw_x': x,
                        'raw_y': y,
                        'status': 'RETRY'}
                    return results_dict                    
                    break
                x[i] = con_ode_sys['robustness'].sol['kp_1']
                y[i] = con_ode_sys['robustness'].sol['kp_2']

                if len(x[i]) > 2 and len(y[i]) > 2:
                    gradients = np.gradient(y[i],x[i])
                    if (gradients[0]/gradients[1]<0.9 or gradients[0]/gradients[1]>1.1): #This means closed curve at beginning. Trim first point
                        x[i] = x[i][2:]
                        y[i] = y[i][2:]
                        PCargs.MaxNumPoints = int(PCargs.MaxNumPoints*2)
                        PCargs.MaxStepSize = PCargs.MaxStepSize/10
                        if attempts == 3:
                            success = True
                        else:
                            success = False
                        continue
                    elif (gradients[-1]/gradients[-2]<0.9 or gradients[-1]/gradients[-2]>1.1):
                        if attempts == 3:
                            success = True
                        else:
                            success = False
                        x[i] = x[i][:-2]
                        y[i] = y[i][:-2]
                        PCargs.MaxNumPoints = int(PCargs.MaxNumPoints * 2)
                        PCargs.MaxStepSize = PCargs.MaxStepSize/10
                        continue


        if (len(x[0])!=0) and (len(x[1])!=0):
            min_x0 = min(x[0])
            min_y0 = min(y[0])

            min_x1 = min(x[1])
            min_y1 = min(y[1])

            if np.argmin(x[0]) not in [0, len(x[0])-1]:

                if (y[0][0] - min_y0)/(x[0][0] - min_x0) > (y[0][-1] - min_y0)/(x[0][-1] - min_x0):#Condition to check if upper curve begins first
                    lower_x = np.sort(x[0][np.argmin(x[0]):])
                    lower_y = np.sort(y[0][np.argmin(x[0]):])
                else:
                    lower_x = np.sort(x[0][:np.argmin(x[0])+1])
                    lower_y = np.sort(y[0][:np.argmin(x[0])+1])
            else:
                lower_x = np.sort(x[0])
                lower_y = np.sort(y[0])


            if np.argmin(x[1]) not in [0, len(x[1])-1]:

                if (y[1][0] - min_y1)/(x[1][0] - min_x1) > (y[1][-1] - min_y1)/(x[1][-1] - min_x1):#Condition to check if upper curve begins first
                    upper_x = np.sort(x[1][:np.argmin(x[1])+1])
                    upper_y = np.sort(y[1][:np.argmin(x[1])+1])
                else:
                    upper_x = np.sort(x[1][np.argmin(x[1]):])
                    upper_y = np.sort(y[1][np.argmin(x[1]):])
            else:
                upper_x = np.sort(x[1])
                upper_y = np.sort(y[1])


        raw_lower_x = lower_x
        raw_lower_y = lower_y
        raw_upper_x = upper_x
        raw_upper_y = upper_y

        #Clearing any duplicate entries in x or y - helps clear up probs with interpolation
        u, indices = np.unique(lower_x, return_index=True)
        lower_x = lower_x[indices]
        lower_y = lower_y[indices]
        u, indices = np.unique(lower_y, return_index=True)
        lower_x = lower_x[indices]
        lower_y = lower_y[indices]

        u, indices = np.unique(upper_x, return_index=True)
        upper_x = upper_x[indices]
        upper_y = upper_y[indices]
        u, indices = np.unique(upper_y, return_index=True)
        upper_x = upper_x[indices]
        upper_y = upper_y[indices]

        try:
            lower_interp = interpolate(lower_x, lower_y, k=3)
            upper_interp = interpolate(upper_x, upper_y, k=3)
        except:
            results_dict = {'params': params_dict,
                'robustness': 0,
                'lower_x': [],
                'lower_y': [],
                'upper_x': [],
                'upper_y': [],
                'raw_lower_x': [],
                'raw_lower_y': [],
                'raw_upper_x': [],
                'raw_upper_y': [],
                'raw_x': x,
                'raw_y': y,
                'status': False}
            return results_dict


        if lower_interp(threshold) < 0:
            lower_x = lower_x[:-1]
            lower_y = lower_y[:-1]

        if upper_interp(threshold) < 0:
            upper_x = upper_x[:-1]
            upper_y = upper_y[:-1]

        if upper_interp(threshold) < threshold and upper_interp(threshold) > 0: #Upper curve does not hit upper threshold -> need to cut short everything at x(kp1)=100

            lower_interp = interpolate(lower_x, lower_y, k=3)
            upper_interp = interpolate(upper_x, upper_y, k=3)

            lower_x = np.linspace(min(lower_x), threshold, 1000)
            lower_y = lower_interp(lower_x)
            upper_x = np.linspace(min(upper_x), threshold, 1000)
            upper_y = upper_interp(upper_x)

        elif lower_interp(threshold) > threshold: #Lower curve hits upper threshold -> Need to cut short everything at y(kp2)=100
            lower_interp = interpolate(lower_y, lower_x, k=3)
            upper_interp = interpolate(upper_y, upper_x, k=3)

            if lower_interp(threshold) > threshold:#Irregular interpolation. Deal by keeping this but truncating appropriately
                lower_y = np.linspace(min(lower_y), threshold, 1000)
                lower_x = lower_interp(np.linspace(min(lower_y), threshold, 1000))
                u, indices = np.unique(lower_x, return_index=True)
                lower_x = lower_x[indices]
                lower_y = lower_y[indices]
                lower_interp = interpolate(lower_x, lower_y, 
                                           k=3)
                lower_x = np.linspace(min(lower_x), threshold, 1000)
                lower_y = lower_interp(lower_x)
            else:

                lower_y = np.linspace(min(lower_y), threshold, 1000)
                lower_x = lower_interp(lower_y)

            if upper_interp(threshold) > lower_interp(threshold):#irregular case
                upper_interp = interpolate(upper_x, upper_y, k=3)
                upper_x = np.linspace(min(upper_x), 5*max(upper_x), 1000)
                upper_y = upper_interp(upper_x)
                u, indices = np.unique(upper_y, return_index=True)
                upper_x = upper_x[indices]
                upper_y = upper_y[indices]
                upper_interp = interpolate(upper_y, upper_x, k=3)  


            upper_y = np.linspace(min(upper_y), threshold, 1000)
            upper_x = upper_interp(upper_y)
            upper_y = np.append(upper_y, [threshold for i in range(100)])
            upper_x = np.append(upper_x, np.linspace(max(upper_x), max(lower_x), 100))

        elif lower_interp(threshold) < threshold: # A normal continuity curve - lower curve intersects x=threshold and upper curve intersects y=threshold
            lower_interp = interpolate(lower_y, lower_x, k=3)
            upper_interp = interpolate(upper_x, upper_y, k=3)


            lower_y = np.linspace(min(lower_y), threshold, 1000)
            lower_x = lower_interp(lower_y)
            u, indices = np.unique(lower_x, return_index=True)
            lower_x = lower_x[indices]
            lower_y = lower_y[indices]
            lower_interp = interpolate(lower_x, 
                                   lower_y, 
                                   k=3)
            lower_x = np.linspace(min(lower_x), threshold, 1000)
            lower_y = lower_interp(lower_x)

            if upper_interp(threshold)<0 and lower_interp(threshold)>0:
                upper_x = np.linspace(min(upper_x), 5*max(upper_x), 1000)
                upper_y = upper_interp(upper_x)
                u, indices = np.unique(upper_y, return_index=True)
                upper_x = upper_x[indices]
                upper_y = upper_y[indices]
                upper_interp = interpolate(upper_y, upper_x, k=3)  
                upper_y = np.linspace(min(upper_y), 2*max(upper_y), 1000)
                upper_x = upper_interp(upper_y)
                u, indices = np.unique(upper_x, return_index=True)
                upper_x = upper_x[indices]
                upper_y = upper_y[indices]
                upper_interp = interpolate(upper_x, upper_y, k=3)
                upper_x = np.linspace(min(upper_x), 5*max(upper_x), 1000)
                upper_y = upper_interp(upper_x)
                u, indices = np.unique(upper_y, return_index=True)
                upper_x = upper_x[indices]
                upper_y = upper_y[indices]
                upper_interp = interpolate(upper_y, upper_x, k=3)
                upper_y = np.linspace(min(upper_y), threshold, 1000)
                upper_x = upper_interp(upper_y)

                if upper_y[-1] < 0:
                    upper_interp = interpolate(raw_upper_x, raw_upper_y, k=3)
                    upper_x = np.linspace(min(raw_upper_x), 5*max(raw_upper_x), 1000)
                    upper_y = upper_interp(upper_x)
                    u, indices = np.unique(upper_y, return_index=True)
                    upper_x = upper_x[indices]
                    upper_y = upper_y[indices]
                    upper_x = np.linspace(min(upper_x), 5*max(upper_x), 1000)
                    upper_y = upper_interp(upper_x)
                    u, indices = np.unique(upper_y, return_index=True)
                    upper_x = upper_x[indices]
                    upper_y = upper_y[indices]
                    upper_interp = interpolate(upper_y, upper_x, k=3) 
                    upper_y = np.linspace(min(upper_y), threshold, 1000)
                    upper_x = upper_interp(upper_y)
                upper_x = np.append(upper_x, np.linspace(max(upper_x), threshold, 100)[1:])
                upper_y = np.append(upper_y, [threshold]*len(np.linspace(max(upper_x), threshold, 100)[1:]))
            else:
                upper_x = np.linspace(min(upper_x), threshold, 1000)
                upper_y = upper_interp(upper_x)
                upper_y = np.array([element if element<threshold else threshold for element in upper_y])

        else:
            results_dict = {'params': params_dict,
                'robustness': 0,
                'lower_x': [],
                'lower_y': [],
                'upper_x': [],
                'upper_y': [],
                'raw_lower_x': [],
                'raw_lower_y': [],
                'raw_upper_x': [],
                'raw_upper_y': [],
                'raw_x': x,
                'raw_y': y,
                'status': True}
            return results_dict


        robustness = np.trapz(upper_y, upper_x) - np.trapz(lower_y, lower_x)
        if robustness < 0:
            robustness = 0



        results_dict = {'params': params_dict,
                        'robustness': robustness,
                        'lower_x': lower_x,
                        'lower_y': lower_y,
                        'upper_x': upper_x,
                        'upper_y': upper_y,
                        'raw_lower_x': raw_lower_x,
                        'raw_lower_y': raw_lower_y,
                        'raw_upper_x': raw_upper_x,
                        'raw_upper_y': raw_upper_y,
                        'raw_x': x,
                        'raw_y': y,
                        'status': True}

        return results_dict
    else:
        results_dict = {'params': params_dict,
            'robustness': 0,
            'lower_x': [],
            'lower_y': [],
            'upper_x': [],
            'upper_y': [],
            'raw_lower_x': [],
            'raw_lower_y': [],
            'raw_upper_x': [],
            'raw_upper_y': [],
            'raw_x': x,
            'raw_y': y,
            'status': True}
        return results_dict

### Vary Params
This functions allows for easy modification of params to supply to above defined functions

In [ ]:
def vary_params(params=['kdeg_1'], value = [1]):
    params_dict =  dict(kp_1=20, kp_2=20,
                        kdiss_1=1, kdiss_2=1,
                        kdeg_1=1, kdeg_2=1,
                        n_1=2, n_2=2)
    for i, param in enumerate(params):
        params_dict[param] = value[i]
    return params_dict

# Upper limit for protein production
Define Threshold for protein production rates here

In [ ]:
threshold = 1000

# Calculations
The following blocks will generate code for each parameter set required. Each block calculates robustness for one set of KDiss values. Only half of all parameter combinations need to be calculated (i.e. KDiss1=x, KDiss2=y is the same as KDiss2=x, KDiss2=y since just the repressors are swapped. Hence the results of the latter case can be inferred from the former)

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (vary_params(['kdeg_1', 'kdeg_2'], 
                                                                           [kdeg_1, kdeg_2]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')

robustnesses = [result['robustness'] for result in results_list_parallel]
indices = [index for index,rob in enumerate(robustnesses[1:-1]) if robustnesses[index]<robustnesses[index-1] and
                                                         robustnesses[index]<robustnesses[index+1]]
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]
prob_params = [results_list_parallel[index]['params'] for index in indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                             for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1'], 
                                                                           [kdeg_1, kdeg_2, 0.1]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')

robustnesses = [result['robustness'] for result in results_list_parallel]
with np.errstate(divide='ignore'):
    indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.1 and 
                                                                 np.divide(robustnesses[i],robustnesses[i-1])>1.1) or
                                                        (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
                                                         np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
                                                       (robustnesses[i-1]!=0 and robustnesses[i+1]!=0 and i%30 not in [0,29])] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]
# indices = indices + [index for index, rob in enumerate(robustnesses) if rob==0]
indices = list(set(indices))
prob_params = [results_list_parallel[index]['params'] for index in indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                             for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1'], 
                                                                           [kdeg_1, kdeg_2, 0.01]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')

robustnesses = [result['robustness'] for result in results_list_parallel]
with np.errstate(divide='ignore'):
    indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.1 and 
                                                                 np.divide(robustnesses[i],robustnesses[i-1])>1.1) or
                                                        (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
                                                         np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
                                                       (robustnesses[i-1]!=0 and robustnesses[i+1]!=0 and i%30 not in [0,29])] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]
indices = indices + [index for index, rob in enumerate(robustnesses) if rob==0]
indices = list(set(indices))
prob_params = [results_list_parallel[index]['params'] for index in indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                             for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
with np.errstate(divide='ignore'):
    indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.1 and 
                                                                 np.divide(robustnesses[i],robustnesses[i-1])>1.1) or
                                                        (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
                                                         np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
                                                       (robustnesses[i-1]!=0 and robustnesses[i+1]!=0 and i%30 not in [0,29])] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]
indices = indices + [index for index, rob in enumerate(robustnesses) if rob==0]
indices = list(set(indices))
prob_params = [results_list_parallel[index]['params'] for index in indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                             for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1'], 
                                                                           [kdeg_1, kdeg_2, 10]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')

robustnesses = [result['robustness'] for result in results_list_parallel]
with np.errstate(divide='ignore'):
    indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.1 and 
                                                                 np.divide(robustnesses[i],robustnesses[i-1])>1.1) or
                                                        (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
                                                         np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
                                                       (robustnesses[i-1]!=0 and robustnesses[i+1]!=0 and i%30 not in [0,29])] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]
indices = indices + [index for index, rob in enumerate(robustnesses) if rob==0]
indices = list(set(indices))
prob_params = [results_list_parallel[index]['params'] for index in indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                             for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
with np.errstate(divide='ignore'):
    indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.1 and 
                                                                 np.divide(robustnesses[i],robustnesses[i-1])>1.1) or
                                                        (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
                                                         np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
                                                       (robustnesses[i-1]!=0 and robustnesses[i+1]!=0 and i%30 not in [0,29])] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]
indices = indices + [index for index, rob in enumerate(robustnesses) if rob==0 and 
                     results_list_parallel[index]['params']['kdeg_1']>np.logspace(-3,3,30)[22]]
indices = list(set(indices))
prob_params = [results_list_parallel[index]['params'] for index in indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                               (param_dict, threshold)
                                                              for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1'], 
                                                                           [kdeg_1, kdeg_2, 100]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')

robustnesses = [result['robustness'] for result in results_list_parallel]
with np.errstate(divide='ignore'):
    indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.1 and 
                                                                 np.divide(robustnesses[i],robustnesses[i-1])>1.1) or
                                                        (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
                                                         np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
                                                       (robustnesses[i-1]!=0 and robustnesses[i+1]!=0 and i%30 not in [0,29])] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]
indices = indices + [index for index, rob in enumerate(robustnesses) if rob==0]
indices = list(set(indices))
prob_params = [results_list_parallel[index]['params'] for index in indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                             for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
new_indices = [(i%30)*30 + int(i/30)
            for i,rob in enumerate(temp_robustnesses[1:-1]) 
            if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.1 and                               
                 np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.1) or                      
                (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
                 np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and         
            (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0 and i%30 not in [0,29])] 
prob_params = [results_list_parallel[index]['params'] for index in new_indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                             for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in new_indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1'], 
                                                                           [kdeg_1, kdeg_2, 500]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')



In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
with np.errstate(divide='ignore'):
    indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.1 and 
                                                                 np.divide(robustnesses[i],robustnesses[i-1])>1.1) or
                                                        (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
                                                         np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
                                                       (robustnesses[i-1]!=0 and robustnesses[i+1]!=0 and i%30 not in [0,29])] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]

temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
indices = indices +  [(i%30)*30 + int(i/30)
            for i,rob in enumerate(temp_robustnesses[1:-1]) 
            if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.1 and                               
                 np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.1) or                      
                (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
                 np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and         
            (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0 and i%30 not in [0,29])] 
indices = indices + [index for index,robustness in enumerate(robustnesses) if robustness == 0]
indices = list(set(indices))

prob_params = [results_list_parallel[index]['params'] for index in indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                             for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1'], 
                                                                           [kdeg_1, kdeg_2, 0.005]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')



In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]

indices = [index for index,robustness in enumerate(robustnesses) if
                    (results_list_parallel[index]['params']['kdeg_1']<1) and 
                     (results_list_parallel[index]['params']['kdeg_2']<0.1)]
indices = list(set(indices))

prob_params = [results_list_parallel[index]['params'] for index in indices]

replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(new_simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                             for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1', 'kdiss_2'], 
                                                                           [kdeg_1, kdeg_2, 0.005, 0.005]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')




In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
indices = []
# with np.errstate(divide='ignore'):
#     indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.05 and 
#                                                                  np.divide(robustnesses[i],robustnesses[i-1])>1.05) or
#                                                         (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
#                                                          np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
#                                                        (robustnesses[i-1]!=0 and robustnesses[i+1]!=0)] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]

#temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
# indices = indices +  [(i%30)*30 + int(i/30)
#             for i,rob in enumerate(temp_robustnesses[1:-1]) 
#             if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.05 and                               
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.05) or                      
#                 (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and  
#             (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0)] 

indices = indices + [index for index,robustness in enumerate(robustnesses) if (robustnesses[index] == 0)]# or
#                            ((results_list_parallel[index]['params']['kdeg_1'] > np.logspace(-3,3,30)[22]) and
#                            (results_list_parallel[index]['params']['kdeg_2'] <10))]

                     
indices = list(set(indices))

prob_params = [results_list_parallel[index]['params'] for index in indices]


replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (param_dict, threshold)
                                                            for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
z = [result['robustness'] for result in results_list_parallel]
param_space = np.logspace(-3,3,30)
trace_list = []
trace_list.append(go.Contour(x=param_space, y=param_space,
                             z=np.array(z).reshape(30,30).transpose(), 
                             colorscale='RdBu', zmin=0, zmax=1e6,
                            contours_coloring='heatmap'))
layout = go.Layout(height=500, width=500,

                   xaxis=dict(title='kdeg_1',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8, type='log',
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(-3,3)),
                  
                  yaxis=dict(title='kdeg_2',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False, type='log',
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              range=(-3, 3),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

fig = go.Figure(data=trace_list, layout=layout)
fig

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1', 'kdiss_2'], 
                                                                           [kdeg_1, kdeg_2, 0.01, 0.005]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')




In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
indices = []
# with np.errstate(divide='ignore'):
#     indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.05 and 
#                                                                  np.divide(robustnesses[i],robustnesses[i-1])>1.05) or
#                                                         (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
#                                                          np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
#                                                        (robustnesses[i-1]!=0 and robustnesses[i+1]!=0)] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]

# temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
# indices = indices +  [(i%30)*30 + int(i/30)
#             for i,rob in enumerate(temp_robustnesses[1:-1]) 
#             if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.05 and                               
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.05) or                      
#                 (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and  
#             (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0)] 

indices = indices + [index for index,robustness in enumerate(robustnesses) if (robustnesses[index] == 0)]# or
#                           ((results_list_parallel[index]['params']['kdeg_1'] in [np.logspace(-3,3,30)[21],np.logspace(-3,3,30)[22],np.logspace(-3,3,30)[23]]) and
#                           (results_list_parallel[index]['params']['kdeg_2'] in [np.logspace(-3,3,30)[21],np.logspace(-3,3,30)[11]]))]

                     
indices = list(set(indices))

prob_params = [results_list_parallel[index]['params'] for index in indices]


replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (param_dict, threshold)
                                                            for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
z = [result['robustness'] for result in results_list_parallel]
param_space = np.logspace(-3,3,30)
trace_list = []
trace_list.append(go.Contour(x=param_space, y=param_space,
                             z=np.array(z).reshape(30,30).transpose(), 
                             colorscale='RdBu', zmin=0, zmax=1e6,
                            contours_coloring='heatmap'))
layout = go.Layout(height=500, width=500,

                   xaxis=dict(title='kdeg_1',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8, type='log',
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(-3,3)),
                  
                  yaxis=dict(title='kdeg_2',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False, type='log',
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              range=(-3, 3),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

fig = go.Figure(data=trace_list, layout=layout)
fig

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1', 'kdiss_2'], 
                                                                           [kdeg_1, kdeg_2, 0.1, 0.005]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')




In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
indices = []
# with np.errstate(divide='ignore'):
#     indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.05 and 
#                                                                  np.divide(robustnesses[i],robustnesses[i-1])>1.05) or
#                                                         (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
#                                                          np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
#                                                        (robustnesses[i-1]!=0 and robustnesses[i+1]!=0)] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]

# temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
# indices = indices +  [(i%30)*30 + int(i/30)
#             for i,rob in enumerate(temp_robustnesses[1:-1]) 
#             if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.05 and                               
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.05) or                      
#                 (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and  
#             (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0)] 

#indices = indices + [index for index,robustness in enumerate(robustnesses) if (robustnesses[index] == 0)]# or
#                           ((results_list_parallel[index]['params']['kdeg_1'] in [np.logspace(-3,3,30)[21],np.logspace(-3,3,30)[22],np.logspace(-3,3,30)[23]]) and
#                           (results_list_parallel[index]['params']['kdeg_2'] in [np.logspace(-3,3,30)[21],np.logspace(-3,3,30)[11]]))]

                     
indices = list(set(indices))

prob_params = [results_list_parallel[index]['params'] for index in indices]


replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                            for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
z = [result['robustness'] for result in results_list_parallel]
param_space = np.logspace(-3,3,30)
trace_list = []
trace_list.append(go.Contour(x=param_space, y=param_space,
                             z=np.array(z).reshape(30,30).transpose(), 
                             colorscale='RdBu', zmin=0, zmax=1e6,
                            contours_coloring='heatmap'))
layout = go.Layout(height=500, width=500,

                   xaxis=dict(title='kdeg_1',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8, type='log',
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(-3,3)),
                  
                  yaxis=dict(title='kdeg_2',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False, type='log',
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              range=(-3, 3),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

fig = go.Figure(data=trace_list, layout=layout)
fig

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1', 'kdiss_2'], 
                                                                           [kdeg_1, kdeg_2, 10, 0.005]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')




In [ ]:
z = [result['robustness'] for result in results_list_parallel]
param_space = np.logspace(-3,3,30)
trace_list = []
trace_list.append(go.Contour(x=param_space, y=param_space,
                             z=np.array(z).reshape(30,30).transpose(), 
                             colorscale='RdBu', zmin=0, zmax=1e6,
                            contours_coloring='heatmap'))
layout = go.Layout(height=500, width=500,

                   xaxis=dict(title='kdeg_1',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8, type='log',
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(-3,3)),
                  
                  yaxis=dict(title='kdeg_2',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False, type='log',
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              range=(-3, 3),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

fig = go.Figure(data=trace_list, layout=layout)
fig

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1', 'kdiss_2'], 
                                                                           [kdeg_1, kdeg_2, 100, 0.005]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')




In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
indices = []
# with np.errstate(divide='ignore'):
#     indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.05 and 
#                                                                  np.divide(robustnesses[i],robustnesses[i-1])>1.05) or
#                                                         (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
#                                                          np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
#                                                        (robustnesses[i-1]!=0 and robustnesses[i+1]!=0)] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]

# temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
# indices = indices +  [(i%30)*30 + int(i/30)
#             for i,rob in enumerate(temp_robustnesses[1:-1]) 
#             if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.05 and                               
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.05) or                      
#                 (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and  
#             (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0)] 

indices = indices + [index for index,robustness in enumerate(robustnesses) if (robustnesses[index] == 0) and
                           ((results_list_parallel[index]['params']['kdeg_1'] <0.02) and
                           (results_list_parallel[index]['params']['kdeg_2'] <0.1))]

                     
indices = list(set(indices))

prob_params = [results_list_parallel[index]['params'] for index in indices]


replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                            for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
z = [result['robustness'] for result in results_list_parallel]
param_space = np.logspace(-3,3,30)
trace_list = []
trace_list.append(go.Contour(x=param_space, y=param_space,
                             z=np.array(z).reshape(30,30).transpose(), 
                             colorscale='RdBu', zmin=0, zmax=1e6,
                            contours_coloring='heatmap'))
layout = go.Layout(height=500, width=500,

                   xaxis=dict(title='kdeg_1',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8, type='log',
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(-3,3)),
                  
                  yaxis=dict(title='kdeg_2',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False, type='log',
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              range=(-3, 3),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

fig = go.Figure(data=trace_list, layout=layout)
fig

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1', 'kdiss_2'], 
                                                                           [kdeg_1, kdeg_2, 500, 0.005]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')




In [ ]:
param_space = np.append(np.geomspace(0.1,92,16), (481.5+92-(np.geomspace(481.5,92,15))[1:]))
num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1', 'kdiss_2'], 
                                                                           [kdeg_1, kdeg_2, 1, 1]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')





In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
indices = []
# with np.errstate(divide='ignore'):
#     indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.05 and 
#                                                                  np.divide(robustnesses[i],robustnesses[i-1])>1.05) or
#                                                         (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
#                                                          np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
#                                                        (robustnesses[i-1]!=0 and robustnesses[i+1]!=0)] 
# indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]

# temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
# indices = indices +  [(i%30)*30 + int(i/30)
#             for i,rob in enumerate(temp_robustnesses[1:-1]) 
#             if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.05 and                               
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.05) or                      
#                 (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and  
#             (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0)] 

indices = indices + [index for index,robustness in enumerate(robustnesses) if (robustnesses[index] == 0)]# and
#                            ((results_list_parallel[index]['params']['kdeg_1'] <2) and
#                            (results_list_parallel[index]['params']['kdeg_2'] >5))]

                     
indices = list(set(indices))

prob_params = [results_list_parallel[index]['params'] for index in indices]


replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (param_dict, threshold)
                                                            for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
z = [result['robustness'] for result in results_list_parallel]
trace_list = []
trace_list.append(go.Contour(x=param_space, y=param_space,
                             z=np.array(z).reshape(30,30).transpose(), 
                             colorscale='RdBu', zmin=0, zmax=1e6,
                            contours_coloring='heatmap'))
layout = go.Layout(height=500, width=500,

                   xaxis=dict(title='kdeg_1',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8, type='log',
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False),
                  
                  yaxis=dict(title='kdeg_2',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False, type='log',
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

fig = go.Figure(data=trace_list, layout=layout)
fig

In [ ]:
param_space = np.logspace(-3, 3, 30)

num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (vary_params(['kdeg_1', 'kdeg_2', 'kdiss_1', 'kdiss_2'], 
                                                                           [kdeg_1, kdeg_2, 0.001, 0.001]), threshold)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')




In [ ]:
z = [result['robustness'] for result in results_list_parallel]
trace_list = []
trace_list.append(go.Contour(x=param_space, y=param_space,
                             z=np.array(z).reshape(30,30).transpose(), 
                             colorscale='RdBu', zmin=0, zmax=1e6,
                            contours_coloring='heatmap'))
layout = go.Layout(height=500, width=500,

                   xaxis=dict(title='kdeg_1',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8, type='log',
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False),
                  
                  yaxis=dict(title='kdeg_2',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False, type='log',
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))

fig = go.Figure(data=trace_list, layout=layout)
fig

In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
indices = []
# with np.errstate(divide='ignore'):
#     indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.05 and 
#                                                                  np.divide(robustnesses[i],robustnesses[i-1])>1.05) or
#                                                         (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
#                                                          np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
#                                                        (robustnesses[i-1]!=0 and robustnesses[i+1]!=0)] 
indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]

# temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
# indices = indices +  [(i%30)*30 + int(i/30)
#             for i,rob in enumerate(temp_robustnesses[1:-1]) 
#             if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.05 and                               
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.05) or                      
#                 (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and  
#             (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0)] 

#indices = indices + [index for index,robustness in enumerate(robustnesses) if (robustnesses[index] == 0)]# and
#                            ((results_list_parallel[index]['params']['kdeg_1'] <2) and
#                            (results_list_parallel[index]['params']['kdeg_2'] >5))]

                     
indices = list(set(indices))

prob_params = [results_list_parallel[index]['params'] for index in indices]


replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness)
                                                              (param_dict, threshold)
                                                            for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
indices = []
# with np.errstate(divide='ignore'):
#     indices = [i for i,rob in enumerate(robustnesses[1:-1]) if ((np.divide(robustnesses[i],robustnesses[i+1])>1.05 and 
#                                                                  np.divide(robustnesses[i],robustnesses[i-1])>1.05) or
#                                                         (np.divide(robustnesses[i],robustnesses[i+1])<1.00 and 
#                                                          np.divide(robustnesses[i],robustnesses[i-1])<1.00)) and 
#                                                        (robustnesses[i-1]!=0 and robustnesses[i+1]!=0)] 
# indices = indices + [index for index, rob in enumerate(robustnesses) if robustnesses[index] > threshold**2]

# temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
# indices = indices +  [(i%30)*30 + int(i/30)
#             for i,rob in enumerate(temp_robustnesses[1:-1]) 
#             if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.05 and                               
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.05) or                      
#                 (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
#                  np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and  
#             (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0)] 

indices = indices + [index for index,robustness in enumerate(robustnesses) if (robustnesses[index] == 0) and
                            ((results_list_parallel[index]['params']['kdeg_1'] in [np.logspace(-3,3,30)[-3], np.logspace(-3,3,30)[-9]]) or
                            (results_list_parallel[index]['params']['kdeg_2'] == 1))]

                     
indices = list(set(indices))

prob_params = [results_list_parallel[index]['params'] for index in indices]


replacement_results = Parallel(n_jobs=num_cores, verbose=5)(delayed(simple_toggle_robustness_repeat)
                                                              (param_dict, threshold)
                                                            for param_dict in prob_params)
results_list_parallel = [result for index, result in enumerate(results_list_parallel) if index not in indices]
results_list_parallel = results_list_parallel + replacement_results
results_list_parallel = sorted(results_list_parallel, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))

In [ ]:
robustnesses = [result['robustness'] for result in results_list_parallel]
indices = []


temp_robustnesses = list(np.array(robustnesses).reshape(30,30).transpose().reshape(1,900)[0]) 
indices = indices +  [(i%30)*30 + int(i/30)
            for i,rob in enumerate(temp_robustnesses[1:-1]) 
            if ((np.divide(temp_robustnesses[i],temp_robustnesses[i+1])>1.01 and                               
                 np.divide(temp_robustnesses[i],temp_robustnesses[i-1])>1.01) or                      
                (np.divide(temp_robustnesses[i],temp_robustnesses[i+1])<1.00 and                       
                 np.divide(temp_robustnesses[i],temp_robustnesses[i-1])<1.00)) and  
            (temp_robustnesses[i-1]!=0 and temp_robustnesses[i+1]!=0)] 